In [60]:
#import libraries
import requests 
from bs4 import BeautifulSoup 
import csv 
import lxml

In [61]:
#get html
URL = "https://www.isaaa.org/gmapprovaldatabase/geneslist/default.asp"
gmo = requests.get(URL) 
print(gmo.content) 

b'\r\n\r\n\r\n<!doctype html>\r\n<html lang="en" class="no-js">\r\n<head>\r\n\t<meta charset="UTF-8">\r\n\t<meta name="viewport" content="width=device-width, initial-scale=1">\r\n\r\n\t<link href="https://fonts.googleapis.com/css?family=Open+Sans|Yellowtail|Montserrat:400,600" rel="stylesheet">\r\n\t<link rel="stylesheet" href="/css/reset.css"> <!-- CSS reset -->\r\n\t<link rel="stylesheet" href="/css/style.css"> <!-- Resource style -->\r\n    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">\r\n  \t\r\n\t<title>Genes List - GM Approval Database | ISAAA.org</title>\r\n    <!-- Facebook Pixel Code -->\r\n<script>\r\n!function(f,b,e,v,n,t,s)\r\n{if(f.fbq)return;n=f.fbq=function(){n.callMethod?\r\nn.callMethod.apply(n,arguments):n.queue.push(arguments)};\r\nif(!f._fbq)f._fbq=n;n.push=n;n.loaded=!0;n.version=\'2.0\';\r\nn.queue=[];t=b.createElement(e);t.async=!0;\r\nt.src=v;s=b.getElementsByTagName(e)[0];\r\ns.parentNode.inser

In [62]:
soup = BeautifulSoup(gmo.content, "lxml")
print(soup.prettify()) 

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://fonts.googleapis.com/css?family=Open+Sans|Yellowtail|Montserrat:400,600" rel="stylesheet"/>
  <link href="/css/reset.css" rel="stylesheet"/>
  <!-- CSS reset -->
  <link href="/css/style.css" rel="stylesheet"/>
  <!-- Resource style -->
  <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet"/>
  <title>
   Genes List - GM Approval Database | ISAAA.org
  </title>
  <!-- Facebook Pixel Code -->
  <script>
   !function(f,b,e,v,n,t,s)
{if(f.fbq)return;n=f.fbq=function(){n.callMethod?
n.callMethod.apply(n,arguments):n.queue.push(arguments)};
if(!f._fbq)f._fbq=n;n.push=n;n.loaded=!0;n.version='2.0';
n.queue=[];t=b.createElement(e);t.async=!0;
t.src=v;s=b.getElementsByTagName(e)[0];
s.parentNode.insertBefore(t,s)}(window,document,'script',
'https://connect.facebo

In [66]:
table = soup.find('div', attrs = {'class':'contenttext'})
table

<div class="contenttext">
<h1>Genes List  </h1>
<table border="0" cellpadding="3" cellspacing="1" width="100%">
<tr class="whitetext">
<td bgcolor="#9494B5"><strong>Gene</strong></td>
<td bgcolor="#9494B5"><strong>Product</strong><strong> and Function</strong></td>
</tr>
<tr>
<td bgcolor="#CFD8D3" colspan="2">Trait: <strong>2,4-D herbicide tolerance</strong></td>
</tr>
<tr class="smalltext">
<td bgcolor="#FAFFE5" valign="top"><a href="/gmapprovaldatabase/gene/default.asp?GeneID=88&amp;Gene=aad-1">
<strong>
          aad-1
          </strong></a><br/>
<strong>Source:</strong>
          synthetic form of the aad-1 gene from <em>Sphingobium herbicidovorans</em>
</td>
<td bgcolor="#FAFFE5" valign="top"><strong>Product:</strong>
        aryloxyalkanoate dioxygenase 1 (AAD-1) protein
        
        <br/>
<strong>Function:</strong>
        detoxifies 2,4-D herbicide by side-chain degradation and degrades the R-enantiomers of aryloxyphenoxypropionate herbicides</td>
</tr>
<tr class="smalltex

In [49]:
xml_soup = BeautifulSoup(gmo.content, 'xml')

In [50]:
print(xml_soup.prettify())

<?xml version="1.0" encoding="utf-8"?>



In [79]:
soup.p.get_attribute_list('contenttext')

[None]

In [16]:
soup.title.string

'Genes List - GM Approval Database | ISAAA.org'

In [69]:
table.string

In [87]:
tr = table.tr
tr

tr.contents

['\n',
 <td bgcolor="#9494B5"><strong>Gene</strong></td>,
 '\n',
 <td bgcolor="#9494B5"><strong>Product</strong><strong> and Function</strong></td>,
 '\n']

In [76]:
table.find_all('td')

[<td bgcolor="#9494B5"><strong>Gene</strong></td>,
 <td bgcolor="#9494B5"><strong>Product</strong><strong> and Function</strong></td>,
 <td bgcolor="#CFD8D3" colspan="2">Trait: <strong>2,4-D herbicide tolerance</strong></td>,
 <td bgcolor="#FAFFE5" valign="top"><a href="/gmapprovaldatabase/gene/default.asp?GeneID=88&amp;Gene=aad-1">
 <strong>
           aad-1
           </strong></a><br/>
 <strong>Source:</strong>
           synthetic form of the aad-1 gene from <em>Sphingobium herbicidovorans</em>
 </td>,
 <td bgcolor="#FAFFE5" valign="top"><strong>Product:</strong>
         aryloxyalkanoate dioxygenase 1 (AAD-1) protein
         
         <br/>
 <strong>Function:</strong>
         detoxifies 2,4-D herbicide by side-chain degradation and degrades the R-enantiomers of aryloxyphenoxypropionate herbicides</td>,
 <td bgcolor="#F7F7F7" valign="top"><a href="/gmapprovaldatabase/gene/default.asp?GeneID=87&amp;Gene=aad-12">
 <strong>
           aad-12
           </strong></a><br/>
 <strong>So

In [81]:
soup.contenttext

In [20]:
print(soup.get_text())








 
 

Genes List - GM Approval Database | ISAAA.org


!function(f,b,e,v,n,t,s)
{if(f.fbq)return;n=f.fbq=function(){n.callMethod?
n.callMethod.apply(n,arguments):n.queue.push(arguments)};
if(!f._fbq)f._fbq=n;n.push=n;n.loaded=!0;n.version='2.0';
n.queue=[];t=b.createElement(e);t.async=!0;
t.src=v;s=b.getElementsByTagName(e)[0];
s.parentNode.insertBefore(t,s)}(window,document,'script',
'https://connect.facebook.net/en_US/fbevents.js');
 fbq('init', '682379405280758'); 
fbq('track', 'PageView');






(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-PJVN35D');






Get updates on COVID-19 research at COVID-19 Resource 






				Menu
			
 

About Us
Programs
Knowledge Center
Resources
GM Approval Database

In [27]:
soup = BeautifulSoup('<td bgcolor="#F7F7F7" valign="top"><strong>Product:</strong> CAld5H enzyme <br/>')
tag = soup.b
type(tag)

NoneType

In [ ]:
for gene in 